In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielens-full-25-million-recommendation-data/ml-25m/movies.csv
/kaggle/input/movielens-full-25-million-recommendation-data/ml-25m/ratings.csv
/kaggle/input/movielens-full-25-million-recommendation-data/ml-25m/genome-tags.csv
/kaggle/input/movielens-full-25-million-recommendation-data/ml-25m/README.txt
/kaggle/input/movielens-full-25-million-recommendation-data/ml-25m/genome-scores.csv
/kaggle/input/movielens-full-25-million-recommendation-data/ml-25m/tags.csv
/kaggle/input/movielens-full-25-million-recommendation-data/ml-25m/links.csv


In [2]:
!pip install scikit-surprise

In [3]:
from surprise import Dataset, Reader
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise import accuracy
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [4]:
ratings_df = pd.read_csv("/kaggle/input/movielens-full-25-million-recommendation-data/ml-25m/ratings.csv")
movies_df = pd.read_csv("/kaggle/input/movielens-full-25-million-recommendation-data/ml-25m/movies.csv")

df = pd.merge(ratings_df, movies_df[['movieId', 'genres']], on = 'movieId', how = 'left')

df

,userId,movieId,rating,timestamp,genres
0,1,296,5.0,1147880044,Comedy|Crime|Drama|Thriller
1,1,306,3.5,1147868817,Drama
2,1,307,5.0,1147868828,Drama
3,1,665,5.0,1147878820,Comedy|Drama|War
4,1,899,3.5,1147868510,Comedy|Musical|Romance
...,...,...,...,...,...
25000090,162541,50872,4.5,1240953372,Animation|Children|Drama
25000091,162541,55768,2.5,1240951998,Animation|Comedy
25000092,162541,56176,2.0,1240950697,Children|Comedy
25000093,162541,58559,4.0,1240953434,Action|Crime|Drama|IMAX


In [5]:
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
mlb = MultiLabelBinarizer()

df['userId'] = user_encoder.fit_transform(df['userId'])
df['movieId'] = movie_encoder.fit_transform(df['movieId'])

df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('genres').str.split('|')), columns = mlb.classes_, index = df.index ))

In [6]:
df

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,292,5.0,1147880044,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,0,302,3.5,1147868817,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,303,5.0,1147868828,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,654,5.0,1147878820,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,0,878,3.5,1147868510,0,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25000090,162540,11359,4.5,1240953372,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
25000091,162540,11925,2.5,1240951998,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
25000092,162540,11972,2.0,1240950697,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
25000093,162540,12216,4.0,1240953434,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [7]:
df.drop(columns = "(no genres listed)", inplace = True)

In [8]:
df

,userId,movieId,rating,timestamp,Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,292,5.0,1147880044,0,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
1,0,302,3.5,1147868817,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,303,5.0,1147868828,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,654,5.0,1147878820,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0,878,3.5,1147868510,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25000090,162540,11359,4.5,1240953372,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
25000091,162540,11925,2.5,1240951998,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
25000092,162540,11972,2.0,1240950697,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
25000093,162540,12216,4.0,1240953434,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0


In [9]:
train_df, test_df = train_test_split(df, test_size = 0.2)
train_df

,userId,movieId,rating,timestamp,Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
10494227,68111,7018,2.5,1535256870,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
21969839,142827,887,3.0,852149363,1,1,0,0,0,0,...,0,0,0,0,1,1,0,1,0,0
9099271,59270,3567,5.0,963420965,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
11269896,73105,1400,2.5,1093491151,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
17249472,111769,2288,2.0,1128320921,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16121854,104512,898,1.5,1089739320,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
23748161,154228,824,2.0,861871240,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
21195005,137716,1513,3.0,974574485,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
5839287,37819,1931,5.0,945223567,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [10]:
reader = Reader(rating_scale = (0.5, 5))
data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()


In [ ]:
model_svd = SVD()
model_svd.fit(trainset)

predictions_svd = model_svd.test(trainset.build_anti_testset())
accuracy.rmse(predictions_svd)

In [ ]:
def get_top_n_recommendations(user_id, n=10):
  user_movies = df[df['userId'] == user_id]['movieId'].unique()
  all_movies = df['movieId'].unique()
  movies_to_predict = list(set(all_movies) - set(user_movies))

  user_movie_pairs = [(user_id, movie_id, 0) for movie_id in movies_to_predict]
  predictions_cf = model_svd.test(user_movie_pairs)

  top_n_recommendations = sorted(predictions_cf, key = lambda x: x.est)[:n]

  for pred in top_n_recommendations:
    predicted_rating = pred.est
    print(predicted_rating)


  top_n_movie_ids = [int(pred.iid) for pred in top_n_recommendations]

  top_n_movies = movie_encoder.inverse_transform(top_n_movie_ids)

  return top_n_movies

In [ ]:

user_id = 221
recommendations = get_top_n_recommendations(user_id)
top_n_movies_titles = movies_df[movies_df['movieId'].isin(recommendations)]['title'].tolist()
print(top_n_movies_titles)